# Projeto

## 1. Configurações de Ambiente

### Adiciona a raíz do diretório ao PYTHONPATH


In [1]:
import geobr

In [2]:
geobr.lookup_muni??

Signature: geobr.lookup_muni(name_muni=None, code_muni=None, verbose=False)
Source:   
def lookup_muni(name_muni=None, code_muni=None, verbose=False):
    """Lookup municipality codes and names.

    By default, it looks for all municipalities. You can also use 'all' to in
    `name_muni` or `code_muni` to get all municipalities.

    Input a municipality NAME or CODE and get the names and codes of
    the municipality's corresponding state, meso, micro, intermediate, and
    immediate regions. You should not select both code_muni and name_muni

    Parameters
    ----------

    name_muni : str, optional
    The municipality name to be looked up

    code_muni: str, optional
    The municipality code to be looked up

    verbose : bool, optional
    by default False

    Returns
    -------
    pd.DataFram
    13 columns identifying the geographies information of that municipality

    Details Only available from 2010 Census data so far

    Raise
    -------
    Exception if code_mun

In [1]:
import sys
import os

# Add the project's root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

### Configuração de gráficos

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Importações

In [5]:
from __future__ import annotations

# 1. Standart Library
import functools
import os
from pathlib import Path
from string import ascii_uppercase
from typing import Any, Iterable, Mapping, Sequence, Tuple
import unicodedata

# 2. Third-party
import basedosdados as bd
from dotenv import load_dotenv
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

# 3. Local Application Modules
import config
from src.roda.pipelines.data_loading import load_geolocated_layers
from src.roda.utils.geo import override_geometry
from roda.utils.cnae import add_cnae_section_letter
from roda.processing.harmonization import schools_to_rais, aggregate_cnes_jobs_by_cep

### Estilo de Gráficos e Formatação de Números

In [ ]:
plt.style.use('Solarize_Light2')
pd.options.display.float_format = '{:,.2f}'.format

### Parâmetros a serem definidos pelo usuário

In [ ]:
# Esta célula define o escopo para ESTA EXECUÇÃO específica do notebook.
# Altere as variáveis na seção "Parâmetros Principais" para customizar a análise.
#

# --- Parâmetros Principais (Altere aqui conforme necessário) ---

# Ano e Mês de referência para as bases de dados que utilizam essa granularidade.
# Estes valores são definidos aqui para esta análise específica.
YEAR = 2023
CNES_MONTH = 10  # Mês de referência para a base do CNES.

# Lista de municípios a serem analisados.
# Formato: uma lista de tuplas, onde cada tupla é (UF, "Nome do Município").
# Os nomes e UFs devem corresponder exatamente às chaves do dicionário
# `MUNICIPIOS` que está definido em seu config.py ou municipalities.json.
#
# Exemplos:
# MUNICIPALITIES_TO_ANALYZE = [("RJ", "Rio de Janeiro")]
# MUNICIPALITIES_TO_ANALYZE = [("SP", "São Paulo"), ("SP", "Campinas")]
# MUNICIPALITIES_TO_ANALYZE = [("MG", "Belo Horizonte"), ("RJ", "Niterói")]
#
MUNICIPALITIES_TO_ANALYZE = [
    ("RJ", "Rio de Janeiro"),
    ("RJ", "Niterói")
]

"""
COL = "etapas_modalidades_oferecidas"
"""

# --- Carregamento de Configurações do Projeto (Geralmente não é necessário editar) ---

# O ID do projeto de faturamento do Google Cloud é carregado do config.py,
# que por sua vez o lê de um arquivo .env na raiz do projeto.
#
# IMPORTANTE: Para que as buscas na Base dos Dados funcionem, você DEVE
# criar um arquivo chamado `.env` na pasta raiz do projeto e colocar seu
# ID de projeto lá, no seguinte formato:
# GCLOUD_PROJECT_ID="seu-id-de-faturamento-aqui"
#
GCLOUD_ID = config.GCLOUD_PROJECT_ID


# --- DERIVAÇÃO DE CÓDIGOS E CAMINHOS (Não é necessário editar abaixo) ---
#
# O código abaixo usa a lista de nomes acima para obter os códigos IBGE
# e definir os caminhos de saída corretos para esta análise.

try:
    # Converte a lista de nomes em uma lista de códigos IBGE
    MUNICIPALITY_CODES = [
        config.MUNICIPIOS[uf][nome] for uf, nome in MUNICIPALITIES_TO_ANALYZE
    ]
except KeyError as e:
    raise KeyError(
        f"Município ou UF não encontrado no config.py: {e}. "
        "Verifique se o nome e a UF estão escritos corretamente."
    )

# Define o diretório de saída para esta análise específica
OUTPUT_DIR = config.RAIS["output_dir"]


# --- Confirmação dos Parâmetros ---
# Imprime um resumo dos parâmetros selecionados para garantir que tudo está correto.

print(f"Análise configurada para o ano: {YEAR}")
print(f"Municípios selecionados: {[nome for _, nome in MUNICIPALITIES_TO_ANALYZE]}")
print(f"Códigos IBGE correspondentes: {MUNICIPALITY_CODES}")
print(f"Resultados serão salvos em: {OUTPUT_DIR}")

[3304557, 3303302]


## 2. 

In [ ]:
camadas_geo = load_geolocated_layers(
    municipalities=MUNICIPALITY_CODES,
    year=YEAR,
    cnes_month=CNES_MONTH,
    billing_project_id=GCLOUD_ID
)

rais_gdf = camadas_geo["rais"]
cnes_gdf = camadas_geo["cnes"]
schools_gdf = camadas_geo["schools"]


rais_gdf.plot()

In [ ]:
def strip_accents(s: str) -> str:
    """
    Remove os acentos de uma string utilizando normalização Unicode.

    Parâmetros:
    ----------
    s : str
        String de entrada que pode conter caracteres acentuados. Se o valor for
        nulo (NaN), retorna uma string vazia.

    Retorna:
    -------
    str
        String sem acentos. Se a entrada for nula, retorna "".

    Exemplo:
    -------
    >>> strip_accents("ação")
    'acao'

    >>> strip_accents("café")
    'cafe'

    >>> strip_accents(None)
    ''
    """

    if pd.isna(s):
        return ""
    return "".join(c for c in unicodedata.normalize("NFD", s)
                    if unicodedata.category(c) != "Mn")

In [ ]:
norm = (schools_gdf[COL].astype("string")
        .fillna("")
        .map(strip_accents)
        .str.lower())

schools_gdf = schools_gdf.assign(
    infantil     = norm.str.contains(r"\binfantil\b").astype("int8"),
    fundamental  = norm.str.contains(r"\bfundamental\b").astype("int8"),
    medio        = norm.str.contains(r"\bmedio\b").astype("int8"),
    profissional = norm.str.contains(r"\bprofissional\b").astype("int8"),
    eja          = norm.str.contains(r"\badultos\b").astype("int8"),
)

## 3. 

In [ ]:
override_cnes_geometry = functools.partial(
    override_geometry,
    id_column="id_estabelecimento_cnes"
)

In [ ]:
# dicionário de (lat, lon) extraídos do Google Maps
"""
overrides = {
    '5042488': (-22.93496303213362, -43.10111846368994),
    '0113891': (-22.82297090020438, -42.97796431846613),
    '0012521': (-22.880157833214476, -43.07872347303554),
    '3784916': (-22.771568234552003, -42.91998438196854),
    '0012599': (-22.881199876918767, -43.078428218305724),
    '2297590': (-22.81838059154707, -43.01194919623843),
    '2291525': (-22.8261384043605, -43.04752398500165),
    '9101039': (-22.84514982733307, -42.95904980987253),
    '7884680': (-22.93934643214391, -43.0586731769422),
    '0105317': (-22.94305755496102, -43.061599214063634),
    '4156390': (-22.7452793538039, -42.83392041918691),
    '9101039': (-22.827399501549497, -43.09059726136488),
}
"""

# aplica somente num único comando:
"""
cnes_gdf = override_cnes_geometry(
    cnes_gdf,
    overrides,
    drop_ids=["9101039"]
)
"""

## 4. 

In [ ]:
rais_gdf = add_cnae_section_letter(rais_gdf, cnae_col="cnae_2", section_col="grupo_cnae")

In [ ]:
rais_gdf = pd.concat([
    rais_gdf,
    aggregate_cnes_jobs_by_cep(cnes_gdf),
    schools_to_rais(schools_gdf)
])